In [ ]:
!pip -q install google-genai google-adk

# Configura a API Key do Google Gemini
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# Instalar Framework ADK de agentes do Google ################################################
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import textwrap
from IPython.display import display, Markdown
import ipywidgets as widgets
import warnings
warnings.filterwarnings("ignore")

# *** Cole aqui as suas funções call_agent, to_markdown, agente_buscador e agente_organizador ***
# Certifique-se de que elas estejam exatamente como na sua versão funcional
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##########################################
# --- Agente Buscador --- #
##########################################

def agente_buscador(topico):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em encontrar perfis do Instagram usando a busca do Google.
        Para o tópico "{topico}", use a ferramenta de busca para encontrar até 5 perfis do Instagram relevantes.
        Sua resposta deve ser uma lista de no máximo 10 nomes de usuários, um por linha, começando com "@".
        Não inclua nenhuma outra informação.
        """,
        description="Agente para encontrar perfis do Instagram no Google",
        tools=[google_search],
    )
    entrada_do_agente_buscador = f"perfis do Instagram sobre {topico}"
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados


##########################################
# --- Agente de Organização --- #
##########################################
def agente_organizador(lista_bruta):
    organizador = Agent(
        name="agente_organizador",
        model="gemini-2.0-flash",
        instruction="""
        Sua tarefa é receber uma lista de perfis do Instagram (um por linha) e formatá-la de maneira clara e numerada.
        Cada perfil deve aparecer em uma linha separada, com um número de ordem crescente (iniciando em 1) antes do nome do perfil, seguido por um ponto e um espaço.
        Certifique-se de que cada nome de perfil comece com o símbolo "@". Se algum nome na lista não tiver "@" no início, adicione-o.
        Não inclua nenhuma outra informação ou texto além da lista numerada com "@" nos perfis.
        """,
        description="Agente responsável por organizar e formatar listas de perfis do Instagram, garantindo o uso de '@'.",
    )
    entrada_do_agente_organizador = f"Lista de perfis:\n{lista_bruta}"
    lista_organizada = call_agent(organizador, entrada_do_agente_organizador)
    return lista_organizada

print("🔍 Achar.IA - Explorador de Perfis ")
display(Markdown("# Descubra Perfis do Instagram"))
display(Markdown("Pesquise por tópicos como \"fitness\", \"culinária\" ou \"arte\" para encontrar perfis que combinam com seus interesses"))
display(Markdown("<br>")) # Adiciona um espaço

# --- Widgets para a busca ---
label_pergunta_topico = widgets.Label(value='Digite um tópico (ex: \'culinária\')')
input_topico = widgets.Text(
    value='',
    placeholder='Digite um tópico (ex: \'culinária\')',
    description='',
    disabled=False,
    layout=widgets.Layout(width='50%') # Ajusta a largura da caixa de texto
)
botao_achar_inicial = widgets.Button(
    description='Buscar',
    disabled=False,
    button_style='primary', # Tenta usar uma cor azulada
    tooltip='Clique para iniciar a busca',
    icon='search' # Adiciona um ícone de lupa (pode não renderizar perfeitamente no Colab)
)
saida_busca_inicial = widgets.Output()

# --- Lógica de interação ---
topico_busca = ""

def exibir_resultados(output_widget, resultados):
    with output_widget:
        output_widget.clear_output()
        if resultados:
            display(Markdown("### Resultados da Busca:"))
            display(Markdown(resultados))
        else:
            display(Markdown("Nenhum resultado encontrado para este tópico."))

def ao_clicar_achar_inicial(b):
    global topico_busca
    saida_busca_inicial.clear_output()
    topico_busca = input_topico.value
    if topico_busca:
        with saida_busca_inicial:
            print(f"Buscando perfis relevantes sobre {topico_busca}...")
            lancamentos_buscados = agente_buscador(topico_busca)

            print("Organizando os resultados...")
            lista_organizada = agente_organizador(lancamentos_buscados)
            exibir_resultados(saida_busca_inicial, lista_organizada)
    else:
        with saida_busca_inicial:
            display(Markdown("Por favor, digite um tópico de busca."))

# Vincula a função ao evento do botão
botao_achar_inicial.on_click(ao_clicar_achar_inicial)

# Exibe os widgets para a busca
display(label_pergunta_topico, input_topico, botao_achar_inicial, saida_busca_inicial)

display(Markdown("<br><br>"))
display(Markdown("© 2025 Achar.IA - Explorador de Perfis • Não afiliado ao Instagram"))